# An Analysis of Political Contributions During the 2020 House of Representatives Election (Part I)

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import re

In this part, you will obtain as much data as you can on the campaign contributions received by each candidate. This data is avaiable through the website https://www.opensecrets.org/. At the end of the project, your group will give a presentation of your findings.

1. Start by scraping the data from the summary page for Tennessee's 2nd District, which is available at https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N.
    * The data that we want is contained in the "Total Raised and Spent" table.
    * Make a DataFrame showing, for each candidate:
        * the candidate's name
        * the candidate's party
        * state
        * district number
        * whether the candidate was an incumbent
        * whether the candidate won the race
        * the total amount raised by that candidate (as a numeric variable)
        * the total amount spent by the candidate (as a numeric variable)

In [3]:
# fetch data from URL
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N'
response = requests.get(URL)
soup = BS(response.text)

# find the raise and spent data table in URL and save it to raise_spent dataframe
raise_spent = pd.read_html(str(soup.find('table', attrs = {'class':'DataTable'})))[0]

In [4]:
raise_spent

,Candidate,Raised,Spent,Cash on Hand,Last Report
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020


In [5]:
# split the candidate column by •
candidate =raise_spent.apply(lambda x: x['Candidate'].split('•'), axis =1)

# is incumbbent?
incumbent = []
for ls in candidate:
    if any('Incumbent' in word for word in ls):
        incumbent.append(True)
    else:
        incumbent.append(False)
        
# is wninner?
winner = []
for ls in candidate:
    if any('Winner' in word for word in ls):
        winner.append(True)
    else:
        winner.append(False)
        
# incumbent and winner columns
raise_spent['Incumbent']=incumbent
raise_spent['Winner'] = winner

In [6]:
# party, state, district, and candidate columns
raise_spent['Party'] = raise_spent['Candidate'].str.extract(('\((\w)\)'))
raise_spent['State']=re.search("[&]\w\w[=](\w\w)(\d\d)",URL).group(1)
raise_spent['District']=re.search("[&]\w\w[=](\w\w)(\d\d)",URL).group(2)
raise_spent['Candidate']=raise_spent['Candidate'].str.extract(("(\w.+)\s[\(]"))

In [7]:
# transform the Raised, Spent, Cash on Hand columns by deleting the $ and ,

def money(column):
    # extract digits from a column to a list
    temp = raise_spent.apply(lambda x: re.findall(r'\d+', x[column]), axis= 1)

    # concatenate digits in a list 
    raised=[]
    for i in range(len(temp)):
        raised.append(''.join(temp[i]))
    return raised
    
raise_spent['Raised']= money('Raised')
raise_spent['Spent']= money('Spent')
raise_spent['Cash on Hand']= money('Cash on Hand')

# change the data type from object to int
raise_spent = raise_spent.astype({'Raised':'int64', 'Spent':'int64', 'Cash on Hand':'int64'})
raise_spent

,Candidate,Raised,Spent,Cash on Hand,Last Report,Incumbent,Winner,Party,State,District
0,Tim Burchett,1336276,878488,593678,12/31/2020,True,True,R,TN,02
1,Renee Hoyos,812784,816793,210,12/31/2020,False,False,D,TN,02


2. Once you have working code for Tennessee's 2nd District, expand on your code to capture all of Tennessee's districts.

In [8]:
# URL for 9 districts of TN
district = []
for i in range (1,10):
    district.append('TN' + str(i).zfill(2))

In [9]:
appended_raise_spent = pd.DataFrame()
for district in district:
    
    URL = f'https://www.opensecrets.org/races/summary?cycle=2020&id={district}&spec=N'
    
    def raise_spent_table(URL):
        # fetch data from URL
        response = requests.get(URL)
        soup = BS(response.text)
        return soup
    
    raise_spent = pd.read_html(str(raise_spent_table(URL).find('table', attrs = {'class':'DataTable'})))[0]
    
        # split the candidate column by •
    candidate =raise_spent.apply(lambda x: x['Candidate'].split('•'), axis =1)

    # is incumbbent?
    incumbent = []
    for ls in candidate:
        if any('Incumbent' in word for word in ls):
            incumbent.append(True)
        else:
            incumbent.append(False)

    # is wninner?
    winner = []
    for ls in candidate:
        if any('Winner' in word for word in ls):
            winner.append(True)
        else:
            winner.append(False)

    # incumbent and winner columns
    raise_spent['Incumbent']=incumbent
    raise_spent['Winner'] = winner

        # party, state, district, and candidate columns
    
    raise_spent['Party'] = raise_spent['Candidate'].str.extract(('\((\w)\)'))
    raise_spent['State']=re.search("[&]\w\w[=](\w\w)(\d\d)",URL).group(1)
    raise_spent['District']=re.search("[&]\w\w[=](\w\w)(\d\d)",URL).group(2)
    raise_spent['Candidate']=raise_spent['Candidate'].str.extract(("(\w.+)\s[\(]"))
    
   # transform the Raised, Spent, Cash on Hand columns by deleting the $ and ,

    def money(column):
        # extract digits from a column to a list
        temp = raise_spent.apply(lambda x: re.findall(r'\d+', x[column]), axis= 1)

        # concatenate digits in a list 
        raised=[]
        for i in range(len(temp)):
            raised.append(''.join(temp[i]))
        return raised

    raise_spent['Raised']= money('Raised')
    raise_spent['Spent']= money('Spent')
    raise_spent['Cash on Hand']= money('Cash on Hand')

    # change the data type from object to int
    raise_spent = raise_spent.astype({'Raised':'int64', 'Spent':'int64', 'Cash on Hand':'int64'})
    appended_raise_spent = appended_raise_spent.append(raise_spent,ignore_index=True)

In [10]:
appended_raise_spent

,Candidate,Raised,Spent,Cash on Hand,Last Report,Incumbent,Winner,Party,State,District
0,Diana Harshbarger,2126946,1869100,257846,12/31/2020,False,True,R,TN,01
1,Blair Nicole Walsingham,140209,134995,5215,10/14/2020,False,False,D,TN,01
2,Tim Burchett,1336276,878488,593678,12/31/2020,True,True,R,TN,02
3,Renee Hoyos,812784,816793,210,12/31/2020,False,False,D,TN,02
4,Chuck Fleischmann,1051653,381411,1880341,12/31/2020,True,True,R,TN,03
5,Meg Gorman,85843,77760,8083,12/31/2020,False,False,D,TN,03
6,Scott Desjarlais,331464,392499,302649,12/31/2020,True,True,R,TN,04
7,Christopher Hale,308731,302996,5735,12/31/2020,False,False,D,TN,04
8,Jim Cooper,936569,1332131,272934,12/31/2020,True,True,D,TN,05
9,John Rose,1050429,625688,454375,12/31/2020,True,True,R,TN,06


3. Once you have working code for all of Tennessee's districts, expand on it to capture all states and districts. The number of representatives each state has can be found in a table on this page: https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120

In [11]:
# fetch data from URL
URL = 'https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120'
response = requests.get(URL)
soup = BS(response.text)

# find the raise and spent data table in URL and save it to raise_spent dataframe
#raise_spent = pd.read_html(str(soup.find('table', attrs = {'class':'DataTable'})))[0]

In [12]:
rep = pd.read_html(str(soup.find('table')))[0]
rep = rep.loc[0:49]

In [13]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}
    
# invert the dictionary
#abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [14]:
rep['abbr'] = rep['state'].map(us_state_to_abbrev)
rep = rep.drop(columns = ['state'])[['abbr','representatives']]

In [15]:
# URL for all districts of all states
district = []
for index in range(50):
    for i in range(1,(rep['representatives'][index]+1)):
        district.append(str(rep['abbr'][index]) + str(i).zfill(2))

In [ ]:
appended_raise_spent = pd.DataFrame()
for district in district:
    
    URL = f'https://www.opensecrets.org/races/summary?cycle=2020&id={district}&spec=N'
    
    def raise_spent_table(URL):
        # fetch data from URL
        response = requests.get(URL)
        soup = BS(response.text)
        return soup
    
    raise_spent = pd.read_html(str(raise_spent_table(URL).find('table', attrs = {'class':'DataTable'})))[0]
    
        # split the candidate column by •
    candidate =raise_spent.apply(lambda x: x['Candidate'].split('•'), axis =1)

    # is incumbbent?
    incumbent = []
    for ls in candidate:
        if any('Incumbent' in word for word in ls):
            incumbent.append(True)
        else:
            incumbent.append(False)

    # is wninner?
    winner = []
    for ls in candidate:
        if any('Winner' in word for word in ls):
            winner.append(True)
        else:
            winner.append(False)

    # incumbent and winner columns
    raise_spent['Incumbent']=incumbent
    raise_spent['Winner'] = winner

        # party, state, district, and candidate columns
    
    raise_spent['Party'] = raise_spent['Candidate'].str.extract(('\((\w)\)'))
    raise_spent['State']=re.search("[&]\w\w[=](\w\w)(\d\d)",URL).group(1)
    raise_spent['District']=re.search("[&]\w\w[=](\w\w)(\d\d)",URL).group(2)
    raise_spent['Candidate']=raise_spent['Candidate'].str.extract(("(\w.+)\s[\(]"))
    
   # transform the Raised, Spent, Cash on Hand columns by deleting the $ and ,

    def money(column):
        # extract digits from a column to a list
        temp = raise_spent.apply(lambda x: re.findall(r'\d+', x[column]), axis= 1)

        # concatenate digits in a list 
        raised=[]
        for i in range(len(temp)):
            raised.append(''.join(temp[i]))
        return raised

    raise_spent['Raised']= money('Raised')
    raise_spent['Spent']= money('Spent')
    raise_spent['Cash on Hand']= money('Cash on Hand')

    # change the data type from object to int
    raise_spent = raise_spent.astype({'Raised':'int64', 'Spent':'int64', 'Cash on Hand':'int64'})
    appended_raise_spent = appended_raise_spent.append(raise_spent,ignore_index=True)

In [ ]:
appended_raise_spent

In [ ]:
appended_raise_spent.to_csv(r'../data/open_secrets.csv', header = True, index= False)